In [3]:
import pandas
import openpyxl
import spacy
from spacy.tokens import DocBin
import string

In [4]:
original_data = pandas.read_excel("anomaly database.xlsx")

In [5]:
for column in original_data:
    print(column)

Customer
Date
Asset
Movement
Notes
Rep Count


In [6]:
note_list_unedited = original_data["Notes"].tolist()

In [7]:
def remove_punctuation(input_string):
        translator = str.maketrans("","", string.punctuation)
        return input_string.translate(translator)

def note_replacer(note_value):
    clean_note = ""  
    note_value = remove_punctuation(note_value)    
    for char in note_value:
        if char.isalpha():
            clean_note += char.lower()
        else:
            clean_note += char
    return clean_note

In [8]:
note_list_edited =[]
for note in note_list_unedited:
    note_list_edited.append(note_replacer(str(note)))

In [9]:
anomaly_column_list=[]
change_column_list=[]
severity_column_list=[]

In [10]:
def fill_anomaly_column(note_list):
    list_to_fill=[]
    nlp = spacy.load("anomaly_best_model/model-best")
    for note in note_list:
        cats_dict = nlp(note).cats
        category = max(cats_dict, key=cats_dict.get)
        list_to_fill.append(category)
    return list_to_fill

In [11]:
def fill_change_column(note_list):
    list_to_fill=[]
    nlp = spacy.load("change_best_model/model-best")
    for note in note_list:
        cats_dict = nlp(note).cats
        category = max(cats_dict, key=cats_dict.get)
        list_to_fill.append(category)
    return list_to_fill

In [13]:
def fill_severity_column(note_list):
    list_to_fill=[]
    nlp = spacy.load("severity_best_model/model-best")
    for note in note_list:
        cats_dict = nlp(note).cats
        category = max(cats_dict, key=cats_dict.get)
        list_to_fill.append(category)
    return list_to_fill

In [14]:
anomaly_column_list=fill_anomaly_column(note_list_edited)

In [24]:
change_column_list=fill_change_column(note_list_edited)

In [25]:
severity_column_list=fill_severity_column(note_list_edited)

In [26]:
new_data = original_data.copy()

In [27]:
new_data['anomaly'] = anomaly_column_list
new_data['change'] = change_column_list
new_data['severity'] = severity_column_list

In [28]:
def fill_status_column(data_frame):
    status_column =[]
    for i in range(len(data_frame)):
        if data_frame["severity"].tolist()[i] == "high":
            status_column.append("check")
        elif data_frame["change"].tolist()[i] == "decrease" and  data_frame["severity"].tolist()[i] != "high":
            status_column.append("report")
        else:
            status_column.append("monitor")
    return status_column
            

In [29]:
status_column = fill_status_column(new_data)

In [30]:
new_data["status"] = status_column

In [31]:
for column in new_data:
    print(column)

Customer
Date
Asset
Movement
Notes
Rep Count
anomaly
change
severity
status


In [155]:
new_data.to_excel('updated anomaly database.xlsx', index = False)